In [1]:
from diffusion import DiffusionEquation
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
import os
import imageio.v2 as imageio

### Create Gif

In [2]:
def create_gif(folder, gif_name, fps=10):
    filenames = glob(os.path.join(folder, "*.png"))
    filenames.sort()
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
    imageio.mimsave(f"../data/gifs/{gif_name}.gif", images, loop=0, fps=fps)

### Create a gif from the images generated from the Heatmap

In [ ]:
def simulate_and_create_heat_map(
    folder: str,
    gif_name: str,
    simulation: DiffusionEquation,
    steps: int,
    save_every: int,
    fps: int = 10,
):

    # Create figs folder
    os.makedirs(folder, exist_ok=True)

    # remove all files inside the folder
    for f in glob(os.path.join(folder, "step*.png")):
        os.remove(f)

    for i in range(steps):
        if i % save_every == 0:
            result = simulation.concentration_matrix

            ax = sns.heatmap(
                result,
                vmin=0,
                vmax=1.0,
                # cmap="Blues",
                fmt="",
                yticklabels=False,
                xticklabels=False,
            )

            ax.tick_params(left=False, bottom=False)
            plt.savefig(os.path.join(folder, f"step_{i:05d}.png"), dpi=300)
            plt.close()

        simulation.omp_step()

    create_gif(folder, gif_name, fps)

### Simulation with a square shaped initial concentration of 1.0 in the center of the domain

In [ ]:
simulation = DiffusionEquation("../build/libDiffusionEquation.so", N=50)

# Create a big concentration square in the middle of the matrix
center = simulation.N // 2
for i in range(-3, 4):
    for j in range(-3, 4):
        simulation.set_initial_concentration({(center + i, center + j): 1.0})


simulate_and_create_heat_map(
    folder="../data/figs/heatmap_center",
    gif_name="simulation_center",
    steps=25000,
    save_every=250,
    simulation=simulation,
    fps=10,
)

### Simulation with a X shaped initial concentration of 1.0

In [7]:
simulation = DiffusionEquation("../build/libDiffusionEquation.so", N=50)

# Create a X shape inside the matrix
for i in range(1, simulation.N - 1):
    N = simulation.N - 1
    simulation.set_initial_concentration({(i, i): 1.0})
    simulation.set_initial_concentration({(i, N - i): 1.0})
    simulation.set_initial_concentration({(N - i, N - i): 1.0})
    simulation.set_initial_concentration({(i, N - i): 1.0})

center = simulation.N // 2
for i in range(-3, 3):
    for j in range(-3, 3):
        simulation.set_initial_concentration({(center + i, center + j): 1.0})

simulate_and_create_heat_map(
    folder="../data/figs_heatmap/heatmap_X",
    gif_name="simulation_X",
    steps=7000,
    save_every=25,
    simulation=simulation,
    fps=30,
)

### Simulation with circle with a concentration of 1.0

In [4]:
# Create a circle of contaminant
center = 50 // 2
radius = 50 // 4  # Adjust the radius as needed
contaminant = {
    (i, j): 1.0
    for i in range(50)
    for j in range(50)
    if abs((((i - center) ** 2 + (j - center) ** 2) ** 0.5) - radius) < 0.5
}
simulation = DiffusionEquation(
    "../build/libDiffusionEquation.so", N=50, initial_concentration_points=contaminant
)


simulate_and_create_heat_map(
    folder="../data/figs_heatmap/heatmap_circle",
    gif_name="simulation_circle",
    steps=7000,
    save_every=25,
    simulation=simulation,
    fps=30,
)